In [1]:
import pandas as pd
import re
from datetime import datetime
from sklearn.mixture import GaussianMixture
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------
# BƯỚC 1: Đọc và xử lý dữ liệu
# ------------------------

# Đọc dữ liệu từ file CSV

df = pd.read_csv(r"E:\Demo\emails.csv")

# Hàm trích thời gian gửi từ trường 'message'
def extract_date(msg):
    match = re.search(r'Date:\s*(.*?)(?=From:)', msg, re.DOTALL)
    return match.group(1).strip() if match else None

# Áp dụng hàm để tạo cột date_str
df['date_str'] = df['message'].apply(extract_date)

# Chuyển về kiểu datetime (ép lỗi sang NaT)
df['datetime'] = pd.to_datetime(df['date_str'], errors='coerce')

# Loại bỏ timezone (nếu có)
def remove_tz(x):
    try:
        return x.astimezone(None).replace(tzinfo=None) if x.tzinfo else x
    except Exception:
        return x

df['datetime'] = df['datetime'].apply(remove_tz)

# Xóa dòng không hợp lệ
df = df.dropna(subset=['datetime'])

# ------------------------
# BƯỚC 2: Trích đặc trưng thời gian
# ------------------------

df['hour'] = df['datetime'].dt.hour + df['datetime'].dt.minute / 60.0

# ------------------------
# BƯỚC 3: Huấn luyện GMM
# ------------------------

X = df['hour'].values.reshape(-1, 1)

gmm = GaussianMixture(n_components=3, random_state=42)
df['cluster'] = gmm.fit_predict(X)

# Thông số mô hình

print("Means:", gmm.means_.flatten())
print("Variances:", gmm.covariances_.flatten())

# ------------------------
# BƯỚC 4: Đánh giá mô hình
# ------------------------

log_likelihood = gmm.score(X) * len(X)
aic = gmm.aic(X)
bic = gmm.bic(X)

print("\n--- Đánh giá mô hình GMM ---")
print(f"Log-Likelihood: {log_likelihood:.2f}")
print(f"AIC: {aic:.2f}")
print(f"BIC: {bic:.2f}")

# ------------------------
# BƯỚC 5: Vẽ biểu đồ kết quả
# ------------------------

plt.figure(figsize=(10,6))
sns.histplot(df['hour'], bins=24, kde=False, color='gray', alpha=0.3, label="Giờ gửi email")

x = np.linspace(0, 24, 1000).reshape(-1, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf * len(df['hour']) * (24 / 30), label='GMM fit', color='red')

plt.title("Phân cụm thời gian gửi email theo GMM")
plt.xlabel("Giờ trong ngày")
plt.ylabel("Số email")
plt.legend()
plt.grid(True)
plt.show()

# ------------------------
# BƯỚC 6: Chọn số cụm tối ưu
# ------------------------

aics = []
bics = []
components = range(1, 10)

for n in components:
    model = GaussianMixture(n_components=n, random_state=42)
    model.fit(X)
    aics.append(model.aic(X))
    bics.append(model.bic(X))

plt.figure(figsize=(10,5))
plt.plot(components, aics, label='AIC', marker='o')
plt.plot(components, bics, label='BIC', marker='o')
plt.xlabel('Số cụm (n_components)')
plt.ylabel('Giá trị AIC/BIC')
plt.title('Chọn số cụm tối ưu cho GMM')
plt.legend()
plt.grid(True)
plt.show()


KeyboardInterrupt: 